<a href="https://colab.research.google.com/github/gracekavemba/-ecommerce-sales-analytics-sql/blob/main/ecommerce_sales_analytics_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🛒 E-commerce Sales Analytics with SQL

This project analyzes an **E-commerce (Online Retail) dataset** using **SQL in Google Colab**.  
The goal is to demonstrate how SQL can be used for **data cleaning, exploration, and business insights**.

### 🔹 Key Steps
1. **Load Dataset** – Import raw transaction data into SQLite.  
2. **Data Cleaning** – Handle missing values, remove duplicates, standardize text, and filter invalid entries.  
3. **SQL Queries** – Extract insights such as:
   - Total revenue
   - Monthly sales trends
   - Best-selling products
   - Top customers by spending
   - Revenue by product category
4. **Visualization** – Plot sales trends and insights for better understanding.  

This project is part of my **SQL portfolio** and shows how raw business data can be transformed into valuable insights.


In [4]:
import pandas as pd
import sqlite3

# Replace with your uploaded filename
df = pd.read_csv("OnlineRetail.csv", encoding='ISO-8859-1')

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [5]:
# Create SQLite database in memory
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Save dataframe into SQL table
df.to_sql('retail', conn, if_exists='replace', index=False)

# Check sample rows
pd.read_sql("SELECT * FROM retail LIMIT 5;", conn)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom


. Total Revenue

In [6]:
query = """
SELECT SUM(Quantity * UnitPrice) AS total_revenue
FROM retail;
"""
pd.read_sql(query, conn)

,total_revenue
0,2.173390e+06


Monthly Sales Trend

In [7]:
query = """
SELECT strftime('%Y-%m', InvoiceDate) AS month,
       SUM(Quantity * UnitPrice) AS monthly_revenue
FROM retail
GROUP BY month
ORDER BY month;
"""
pd.read_sql(query, conn)


,month,monthly_revenue
0,None,2.173390e+06


Best-Selling Products

In [8]:
query = """
SELECT Description, SUM(Quantity) AS total_sold
FROM retail
GROUP BY Description
ORDER BY total_sold DESC
LIMIT 10;
"""
pd.read_sql(query, conn)

,Description,total_sold
0,WORLD WAR 2 GLIDERS ASSTD DESIGNS,13565.0
1,PACK OF 72 RETROSPOT CAKE CASES,12056.0
2,WHITE HANGING HEART T-LIGHT HOLDER,11957.0
3,JUMBO BAG RED RETROSPOT,10608.0
4,SMALL POPCORN HOLDER,8874.0
5,PACK OF 12 LONDON TISSUES,8843.0
6,MINI PAINT SET VINTAGE,7577.0
7,ASSORTED COLOUR BIRD ORNAMENT,7551.0
8,BROCADE RING PURSE,7322.0
9,GROW A FLYTRAP OR SUNFLOWER IN TIN,7252.0


Top Customers by Spending

In [13]:
query = """
CREATE TABLE retail_clean AS
SELECT *
FROM retail
WHERE CustomerID IS NOT NULL;
"""

cursor.execute(query)

# Verify the new table by selecting a few rows
pd.read_sql("SELECT * FROM retail_clean LIMIT 5;", conn)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [9]:
query = """
SELECT CustomerID, SUM(Quantity * UnitPrice) AS total_spent
FROM retail
GROUP BY CustomerID
ORDER BY total_spent DESC
LIMIT 10;
"""
pd.read_sql(query, conn)

,CustomerID,total_spent
0,NaN,379032.58
1,14646.0,57821.02
2,18102.0,39766.09
3,12415.0,37674.04
4,14156.0,32935.62
5,15061.0,24346.02
6,15769.0,23181.48
7,15749.0,22998.40
8,17450.0,21616.66
9,17511.0,21157.10
